In [5]:
from pathlib import Path
import os
import json
import copy
import numpy as np
import pandas as pd
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics


In [6]:
# from ..tools.preprocess import Preprocessor
import sys
sys.dont_write_bytecode = True
sys.path.append('../')
from tools.maneger import DataManager
# from utterance.feature import Feature
from feature import Feature

/usr/lib/python3/dist-packages/requests/__init__.py:80: RequestsDependencyWarning: urllib3 (1.26.5) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [8]:
def json2data(path):
    cols = ["text", "label", "subLabel"]
    df = pd.DataFrame(index=[], columns=cols)
    files = os.listdir(path)
    for cop in files:
        if "." not in cop:
            continue
        with open(path+cop, "r") as f:
            json_data = json.load(f)
            mode = cop.split(".")[0]
            max_ = 300
            for i, data in enumerate( json_data[mode] ) :
                if i > max_:
                    break
                text = data["data"]
                label = data["label"][0]
                # if label == "plain":
                #     break
                subLabel = ""
                df = df.append(pd.DataFrame([text, label, subLabel], index = cols).T)
    df.reset_index(inplace=True, drop=True)
    return df

In [10]:
label_list = "YN WH please proposal plain".split()
label_dict = dict( zip(label_list, range(len(label_list))) )

def extract_X_y(df):
    X = []
    y = []
    for te, la in zip(df.text, df.label):
        X.append(te)
        # if la == "WH":
        #     y.append(0)
        # else:
        #     y.append(label_dict[la])
        y.append(label_dict[la])
    return X, y
    

In [12]:
corpus_root = "../../corpus"
# name = "question/short"
name = "question"
data_path = "/".join([corpus_root, name]) + "/"
data_path

'../../corpus/question/'

In [16]:
df = json2data(data_path)
df


,text,label,subLabel
0,メニューを見せていただけますか？,please,
1,おいでいただけますか？,please,
2,マッシュポテトをもらえますか？,please,
3,伝言を預かっていただけますか？,please,
4,ご一緒しませんか？,please,
...,...,...,...
1236,メンバーになりたいかい？,YN,
1237,明日スタートなさいますか？,YN,
1238,トレイがいりますか？,YN,
1239,2階も見たいですか？,YN,


In [19]:
X, y = extract_X_y(df)

In [22]:
X_train_str, X_test_str, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=5)

In [25]:
F_path = "../X_y_data/response/"
F_name = "typeClassify_F.dill"
featureM = DataManager(F_path, format_="dill")

# if featureM.is_exist(F_name):
#     F = featureM.load_data(F_name)
# else:
#     F = Feature()
#     F.make_features(X_train_str)
#     featureM.save_data(F_name, F)

F = Feature()
F.make_features(X_train_str)
featureM.save_data(F_name, F)


300
success save : ../X_y_data/response/typeClassify_F.dill


In [28]:
print(F.feature_num)
X_train = []
X_test = []
for i, x_t_str in enumerate( X_train_str ):
    x = F.featurization(x_t_str)
    X_train.append(x)
for i, x_t_str in enumerate( X_test_str ):
    x = F.featurization(x_t_str)
    X_test.append(x)
X_train = np.array(X_train)
X_test = np.array(X_test)

31696


In [31]:
lr = LogisticRegression(solver='sag', max_iter=10000)
lr.fit(X_train, y_train)

LogisticRegression(max_iter=10000, solver='sag')

In [34]:
y_pred = lr.predict(X_test)

In [35]:
print(label_dict)
for y_p, x_s in zip(y_pred[:30], X_test_str[:30]):
    print("{0} : {1}".format(y_p, x_s))

{'YN': 0, 'WH': 1, 'please': 2, 'proposal': 3, 'plain': 4}
0 : お名前は何とおっしゃいますか？
4 : 先週いっしょにつくったでしょ。
3 : 同じ美容院へ行ったらどう、お父さん?
2 : 本題に入ってくれないか？
2 : 日曜日に、映画にごいっしょしていただけますか？
3 : 青森にいるあなたのお兄さんのヒデキを訪ねるのはどうかしら？
0 : これって本当にあなたなの？
3 : 日本語を習ってみたら？
4 : 申し訳ないけど、知りませんね。
0 : ぼくといっしょに取り組まないか？
0 : そんなことが信じられるだろうか？
1 : エミ叔母ちゃんは何時に来る予定なの？
4 : カイはすごく欲しかったんだ。
0 : もう質問は終わったかい？
2 : それはどんなものだったのか私に説明してもらえますか？
3 : 私といっしょに食卓の用意をしてくれない？
3 : チケットを取りましょうか？
3 : いっしょに散歩に出かけましょうか？
2 : 皆さんに自己紹介をお願いできますか？
0 : そこにはゲームセンターはある？
4 : 「さあ入って、パーティーはちょうど始まるところ。コートを預かりましょうか？」
0 : こちらに座りましょうか？
4 : わたしの家で課題をしましょうよ。
2 : 伝言を預かっていただけますか？
3 : よく眠れた？
3 : どうしてロボコーパスが戦わなくちゃいけないんだ？
0 : 写真をとることはできるかな？
2 : 質問してもいいですか？
4 : ７月25日だよ。
3 : 日本の古い怪談映画の特別プログラムを上映しない？


In [36]:
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred))
# print('precision = ', precision_score(y_true=y_test, y_pred=y_pred))
# print('recall = ', recall_score(y_true=y_test, y_pred=y_pred))
# print('f1 score = ', f1_score(y_true=y_test, y_pred=y_pred))

confusion matrix = 
 [[69  5  7  8  2]
 [12 13  0  6  0]
 [ 5  0 46  7  2]
 [ 6  1  7 84  3]
 [ 0  0  0  1 89]]
accuracy =  0.806970509383378


In [37]:
from sklearn.ensemble import AdaBoostClassifier
clf = AdaBoostClassifier()


In [38]:
model_path = "../models/response/"
model_name = "typeClassify_M.pickle"
modelM = DataManager(model_path)
print(model_name)

typeClassify_M.pickle


In [39]:
modelM.save_data(model_name, lr)

success save : ../models/response/typeClassify_M.pickle


In [40]:
# accuracy =  0.7913907284768212
# accuracy =  0.8326446280991735
# accuracy =  0.8367768595041323

In [41]:
F2_path = "../X_y_data/response/"
F2_name = "typeClassify_F2.dill"
featureM2 = DataManager(F2_path, format_="dill")

F2 = Feature()
F2.make_features(X_train_str[:3])
featureM2.save_data(F2_name, F2)

300
success save : ../X_y_data/response/typeClassify_F2.dill


In [42]:
F2.featurization("そうですね．最近熱いですから")

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 1., 0., 0., 1., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 1., 1., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])

In [43]:
from pyknp import Juman
from sentence_transformers import SentenceTransformer
import scipy.spatial
model_path = "/home/yamada/Downloads/training_bert_japanese"
Nmodel = SentenceTransformer(model_path, show_progress_bar=False)
emb_dim = Nmodel.encode(["お辞儀をしている男性会社員"])[0].shape[0]

In [44]:
X_bert = Nmodel.encode(X_train_str)
X_bert_test = Nmodel.encode(X_test_str)

In [45]:
from sklearn.svm import SVC
from sklearn.multiclass import OneVsRestClassifier
est = SVC(kernel="rbf", random_state=2, C=1, gamma=0.01)
clf = OneVsRestClassifier(est)

In [46]:
clf.fit(X_bert, y_train)

OneVsRestClassifier(estimator=SVC(C=1, gamma=0.01, random_state=2))

In [47]:
y_pred_b = clf.predict(X_bert_test)

In [48]:
print('confusion matrix = \n', confusion_matrix(y_true=y_test, y_pred=y_pred_b))
print('accuracy = ', accuracy_score(y_true=y_test, y_pred=y_pred_b))

confusion matrix = 
 [[40  0 13 14 24]
 [11  7  2  4  7]
 [ 8  0 33  9 10]
 [16  2 12 50 21]
 [ 3  1  0  2 84]]
accuracy =  0.5737265415549598
